<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/acme.jpeg width=100 align="left">

# Database Engine - MariaDB/MySQL Dynamic Credentials
---
Dynamic Secrets are generated when they are accessed.  In other words, Dynamic Secrets do not exist until they are read.
Vault has built-in revocation mechanism.
Database Secrets engine:
* Cassandra
* Elasticsearch
* Influxdb
* HanaDB
* MongoDB
* MSSQL
* MySQL/MariaDB
* PostgreSQL
* Oracle

## Business Value of Capability
We use dynamic secrets to allow applications to get dynamic creds to access databases.  
### Setup
---

In [3]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_TOKEN=root
export VAULT_SKIP_VERIFY=true

In [ ]:
vault login root

In [ ]:
vault status

In [ ]:
vault -version

**Create a database**

In [ ]:
docker create \
  --name=mariadb \
  -e PUID=1000 \
  -e PGID=1000 \
  -e TZ=Asia/Singapore \
  -e MYSQL_ROOT_PASSWORD=password \
  -e MYSQL_DATABASE=vaultdemodb `#optional` \
  -e MYSQL_USER=tio `#optional` \
  -e MYSQL_PASSWORD=password `#optional` \
  -p 3306:3306 \
  --restart unless-stopped \
  linuxserver/mariadb

**Start the database**

In [ ]:
docker start mariadb

**Execute Two Commands Below in Jupyter Terminal Tab**

docker exec -it mariadb /bin/bash

mysql -u root -ppassword << EOF
GRANT ALL PRIVILEGES ON *.* TO 'vaultadmin'@'%' IDENTIFIED BY 'vadmin' WITH GRANT OPTION;
FLUSH PRIVILEGES;
EOF

In [10]:
docker exec -it mariadb mysql -u root -p'password' \
  -e "GRANT ALL PRIVILEGES ON *.* TO 'vaultadmin'@'%' IDENTIFIED BY 'vadmin' WITH GRANT OPTION; FLUSH PRIVILEGES;"


**Display existing database users**

In [11]:
docker exec -it mariadb mysql -u vaultadmin -p'vadmin' -e "select user from mysql.user;"

+-------------+
| User        |
+-------------+
| root        |
| tio         |
| vaultadmin  |
| mariadb.sys |
+-------------+


### Demo
---

### Enable Database secrets engine

In [12]:
vault secrets enable -path=mdb1 database

Error enabling: Error making API request.

URL: POST http://127.0.0.1:8200/v1/sys/mounts/mdb1
Code: 400. Errors:

* path is already in use at mdb1/


**Configure it with the correct plugin, connection string and allowed_roles**

You can specify `root_rotation_statements` when writing to the path <database/config/<connection>.  For example
```  
   root_rotation-statements="ALTER USER {{username}}'@'localhost' IDENTIFIED BY '{{password}}';"
```  
    

In [6]:
vault write mdb1/config/mysql \
    plugin_name=mysql-legacy-database-plugin \
    connection_url="vaultadmin:vadmin@tcp(127.0.0.1:3306)/" \
    allowed_roles="readonly" \
    username="hashicorp" \
    password="Password123"
    
    
# vault write mdb1/config/mysql \
#    plugin_name=mysql-legacy-database-plugin \
#    connection_url="{{username}}:{{password}}@tcp(127.0.0.1:3306)/" \
#    allowed_roles="readonly" \
#    username="vaultadmin" \
#    password="vadmin"


**Create one or more Vault roles with TTLs and SQL that specify permissions**

In [20]:
vault write mdb1/roles/readonly \
    db_name=mysql \
    creation_statements="CREATE USER '{{name}}'@'%' IDENTIFIED BY '{{password}}';GRANT SELECT ON *.* TO '{{name}}'@'%';" \
    default_ttl="5m" \
    max_ttl="24h"

Success! Data written to: mdb1/roles/readonly


In [ ]:
vault write -force mdb1/rotate-root/config/mysql

**Get Dyamic Database credential by reading from the Vault role**

In [21]:
vault read mdb1/creds/readonly

Key                Value
---                -----
lease_id           mdb1/creds/readonly/s1TCW8LXq2EvH0e3AdC9Z1Q3
lease_duration     5m
lease_renewable    true
password           PVCpstrbr-o-hZs2RC76
username           v_token_read_HoX


**Display database users** (note new user added by Vault)

In [27]:
docker exec -it mariadb mysql -u vaultadmin -p'vadmin' -e "select user from mysql.user;"

+-------------+
| User        |
+-------------+
| root        |
| tio         |
| vaultadmin  |
| mariadb.sys |
+-------------+


**List the Vault leases**

In [26]:
vault list sys/leases/lookup/mdb1/creds/readonly/

Keys
----
BD8UYffxzkq7K09iTvR6jzuB


**Details on a lease, e.g. the remaining lifetime of the credential** COPY LEASE ABOVE TO REPLACE END OF THIS LINE 

In [25]:
vault write sys/leases/lookup lease_id=mdb1/creds/readonly/s1TCW8LXq2EvH0e3AdC9Z1Q3

Error writing data to sys/leases/lookup: Error making API request.

URL: PUT http://127.0.0.1:8200/v1/sys/leases/lookup
Code: 400. Errors:

* invalid lease


**Revoke a specific Vault lease**   
Vault automatically deletes expired credentials but if credentials are compromised, you can revoke them immediately.

In [17]:
vault lease revoke mdb1/creds/readonly/8JV4Ic0e9zyH2zYpHN6AtyYB

All revocation operations queued successfully!


In [ ]:
vault write sys/leases/revoke lease_id="<lease_id>"

**Revoke all Vault leases**

In [18]:
vault lease revoke -prefix=true mdb1/creds/readonly

All revocation operations queued successfully!


**Renew credentials**

In [ ]:
vault write sys/leases/renew lease_id="<lease_id>"

**Disable Database Secret Engine**

In [ ]:
bvault secrets disable database/ 

### Cleanup
---

**Shutdown Docker**

In [ ]:
docker stop mariadb

**Remove the container**

In [ ]:
docker rm mariadb

**END**